In [9]:
# Numerical integration
import numpy as np
from timeit import default_timer as timer

def trap_rule(fun,a,b,n):
    """
    perform trapezoidal rule quadrature
    
    Args:
        fun: name of integrand function
        a, b: numerical integration limits (a<b)
        n: integer number of quadrature subintervals
    Returns:
        h: spacing between quadrature points
        quad: trapezoidal rule quadrature value    
    """
    x = np.linspace(a,b,n+1)
    h = (b-a)/n
    f = np.zeros(n+1)
    for i in range (n+1):
        f[i] = fun(x[i])

    quad = 0.5*h*(2*np.sum(f) - f[0] - f[-1])

    return h, quad


if __name__ == '__main__':
    n = 8 #maximum exponent of h
    a = 0.
    b = np.pi
    e_trap = np.zeros(n) #initialize array to store error values
    h_vals = np.zeros(n) #initialize array to store spacing values
    n_vals = np.zeros(n) #initialize array to store number of points
    times = np.zeros(n) #initialize array to store computation times
    print("h           Absolute Error    Time")
    for i in range(n):
        m = 10**(i+1)
        start = timer()
        h, est =  trap_rule(np.sin,a,b,m)
        end = timer()
        times[i] = (end - start) # Time in seconds
        error = np.abs(est) # exact - est?
        e_trap[i] = error
        h_vals[i] = h
        n_vals[i] = m+1
        print('{:1.0e} {: 1.14f} {: 1.3e}'.format(h,e_trap[i], times[i]))

h           Absolute Error    Time
3e-01  1.98352353750945  3.920e-04
3e-02  1.99983550388744  8.902e-04
3e-03  1.99999835506566  3.298e-03
3e-04  1.99999998355066  3.261e-02
3e-05  1.99999999983551  2.939e-01
3e-06  1.99999999999835  2.715e+00
3e-07  1.99999999999998  2.535e+01
3e-08  2.00000000000000  2.407e+02


In [ ]:
# Serial implementation of simpsons rule

def simpson(f,a,b,n):
    """
    perform trapezoidal rule quadrature
    
    Args:
        fun: name of integrand function
        a, b: numerical integration limits (a<b)
        n: integer number of quadrature panels
    Returns:
        h: spacing between quadrature points
        quad: trapezoidal rule quadrature value    
    """
    h = (b-a)/(2*n)
    x = a + h
    quad = f(a) + 4*f(x) + f(b)
    sum = 3
    for i in range (1,n):
        x += h
        quad += 2*f(x)
        x += h
        quad += 4*f(x)
        sum += 2
    return h, (h/3)*quad

In [ ]:
import os
os.environ['NUMBA_ENABLE_CUDASIM'] = '1'

from numba import cuda, float32
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
import numpy as np
import matplotlib.pyplot as plt

N = 2
M = 8
iters = 2**M

@cuda.jit
def compute_pi(rng_states, iterations, out):
    """Find the maximum value in values and store in result[0]"""
    thread_id = cuda.grid(1)

    # Compute pi by drawing random (x, y) points and finding what
    # fraction lie inside a unit circle
    inside = 0
    scale = 1.1
    for i in range(iterations):
        x = scale * xoroshiro128p_uniform_float32(rng_states, thread_id)
        y = scale * xoroshiro128p_uniform_float32(rng_states, thread_id)
        if x**N + y**N <= 1.0:
            inside += 1

    out[thread_id] = 4.0 *scale*scale * inside / iterations

threads_per_block = 64
blocks = 64
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=1)
out = np.zeros(threads_per_block * blocks, dtype=np.float32)

compute_pi[blocks, threads_per_block](rng_states, iters, out)

monte_pi = out.mean()
print('pi:', monte_pi)
print('relative error = ', np.abs((np.pi-monte_pi)/np.pi))
print('evaluation count: %3.2e' %(threads_per_block * blocks * iters)) 

In [ ]:
# Example for class on grid quadrature
import numpy as np
import math
from numba import cuda, float32, jit, int32
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
#from plotUtils import *

EPS = 1.e-8
N = 512

@cuda.reduce
def sum_reduce(a,b):
    return a + b

@cuda.jit(device=True)
def chi(f):
    return 1 if f<0 else 0

@cuda.jit(device=True)
def f(x,y,r):
    return math.sqrt(x*x + y*y) -r #circle
    #return max(max(abs(x), abs(y)) - 1., -((x-r)**2 + (y-r)**2 - r**2)) #cut block

TPB = 8
RAD = 1

@cuda.jit
def sampleKernel(d_u, d_x, d_y, r):
    i,j = cuda.grid(2)
    dims = d_u.shape
    if i >=dims[0] or j >= dims[1]:
        return
    d_u[i,j] = f(d_x[i], d_y[j],r)

@cuda.jit
def integrateKernel(d_out, d_u, d_x, d_y, p):
    i,j = cuda.grid(2)
    dims = d_u.shape
    if i <RAD or j < RAD or i >= dims[0]-RAD or j >= dims[1]-RAD:
        return
    
    west, east = d_u[i-1,j], d_u[i+1, j]
    south, north = d_u[i, j-1], d_u[i, j+1]

    dfdx, dchidx = (east - west), (chi(east) - chi(west))
    dfdy, dchidy = (north-south), (chi(north) - chi(south))

    denom2 = dfdx * dfdx + dfdy * dfdy
    numer = dchidx * dfdx + dchidy * dfdy

    if p == 2: # I_zz
        g = d_x[i]**2 + d_y[j]**2
    elif p == 1: # I_xx
        g = d_x[i]**2
    else: # length
        g = 1

    if denom2 < EPS or 0 == numer:
        d_out[i,j] = 0
    else:
        d_out[i,j] = g *  numer/math.sqrt(denom2)

def integrateWrapper(x,y,r,p):
    d_x = cuda.to_device(x)
    d_y = cuda.to_device(y)
    dims = (x.size, y.size)
    d_u = cuda.device_array(dims, dtype=np.float32)
    gridSize = [(dims[0]+ TPB-1)//TPB, (dims[1]+TPB-1)//TPB]
    blockSize = [TPB, TPB]
    sampleKernel[gridSize, blockSize](d_u, d_x, d_y, r)
    d_out = cuda.device_array(dims, dtype=np.float32)
    integrateKernel[gridSize, blockSize](d_out, d_u, d_x, d_y, p)

    return d_out.copy_to_host()


if __name__ == '__main__':
    r = 1.0 # geometric parameter radius of circle
    m = 1.1 # coordinate bounds x and y lie between -m and m
    delta = 2*m/(N-1) # grid spacing
    x = np.linspace(-m, m, N)
    y = np.linspace(-m, m, N)
    p = 1 # set p: 0 is length, 1 is I_xx, 2 is I_zz
    properties = ['path length', 'I_xx', 'I_zz']
    print("Integral Properties to be evaluated: " + properties[p])

    out = integrateWrapper(x,y,r,p)

    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.plot_surface(x,y, out.T, cmap=plt.cm.jet)
    # ax.set_xlabel('x')
    # ax.set_ylabel('y')
    # ax.set_zlabel('u')
    ax.set_title('Unsteady Heat Conduction')
    plt.show()


    # plot3d(-out.T,x,y)
    # arraycountorplot(out.T,x,y,levels =[-1.,-0.75,-0.5,-0.25,0])
    plt.contour(x,y,out.T,levels =[-1.,-0.75,-0.5,-0.25,0])
    plt.show()

    print("With ", N*N/1e6, " million points; property values = ", -(delta/2.)*sum_reduce(out.flatten()))
